In [7]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
import glob

import pandas as pd

In [9]:
big_data = pd.DataFrame()
files = glob.iglob('backtests/binance/' + '**/backtest_result.txt', recursive=True)
i = 0
for filename in files:
    #print(f'parsing : {filename}')
    with open(filename, 'r') as file:
        data = file.read().split('\n')
        d = {'file': 'C:\\AgodaGit\\passivbot\\'+filename}
        for l in data:
            l = l.split('|')
            if len(l) == 4:
                key = l[1].strip()
                value = l[2].strip()

                if key == 'Passivbot Version':
                    d['v'] = value
                if key == 'Symbol':
                    d['s'] = value
                if key == 'Total gain':
                    d['tg_%'] = float(value.replace('%', '').strip())
                if key == 'Average daily gain':
                    d['adg_%'] = float(value.replace('%', '').strip())
                if key == 'Closest bankruptcy':
                    d['bnkr_%'] = float(value.replace('%', '').strip())
                if key == 'Long':
                    d['long'] = bool(value) or False
                if key == 'Short':
                    d['short'] = bool(value) or False
                if key == 'Lowest equity/balance ratio':
                    d['ebr'] = float(value.replace('%', '').strip())

        big_data = big_data.append(d, ignore_index=True)

big_data

C:\Users\skhan\AppData\Local\Temp/ipykernel_53572/1044504690.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_index=True)
C:\Users\skhan\AppData\Local\Temp/ipykernel_53572/1044504690.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_index=True)
C:\Users\skhan\AppData\Local\Temp/ipykernel_53572/1044504690.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_index=True)
C:\Users\skhan\AppData\Local\Temp/ipykernel_53572/1044504690.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_inde

,file,v,s,ebr,long,short,adg_%,bnkr_%,tg_%
0,C:\AgodaGit\passivbot\backtests/binance\1INCHU...,5.3,1INCHUSDT,0.909573,True,NaN,NaN,NaN,NaN
1,C:\AgodaGit\passivbot\backtests/binance\1INCHU...,5.3,1INCHUSDT,0.114573,True,True,NaN,NaN,NaN
2,C:\AgodaGit\passivbot\backtests/binance\1INCHU...,5.3,1INCHUSDT,0.838780,NaN,True,NaN,NaN,NaN
3,C:\AgodaGit\passivbot\backtests/binance\1INCHU...,5.3,1INCHUSDT,0.952475,True,NaN,NaN,NaN,NaN
4,C:\AgodaGit\passivbot\backtests/binance\1INCHU...,5.3,1INCHUSDT,0.242079,True,True,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
461,C:\AgodaGit\passivbot\backtests/binance\XTZUSD...,5.3,XTZUSDT,0.847562,True,True,NaN,NaN,NaN
462,C:\AgodaGit\passivbot\backtests/binance\XTZUSD...,5.3,XTZUSDT,0.841806,NaN,True,NaN,NaN,NaN
463,C:\AgodaGit\passivbot\backtests/binance\XTZUSD...,5.3,XTZUSDT,0.972171,True,NaN,NaN,NaN,NaN
464,C:\AgodaGit\passivbot\backtests/binance\XTZUSD...,5.3,XTZUSDT,0.847562,True,True,NaN,NaN,NaN


Now filter the dataframe

In [12]:
allowed_symbols = ["ETHUSDT", "XRPUSDT", "1000SHIBUSDT", "DOGEUSDT", "ADAUSDT",
                   "BNBUSDT", "MATICUSDT", "DOTUSDT", "SANDUSDT", "FTMUSDT",
                   "LTCUSDT", "LINKUSDT", "MANAUSDT", "ETCUSDT", "EOSUSDT",
                   "ATOMUSDT", "FILUSDT", "ICPUSDT", "ALICEUSDT", "LRCUSDT",
                   "BCHUSDT", "SUSHIUSDT", "XTZUSDT", "THETAUSDT", "CRVUSDT",
                   "ALGOUSDT", "CHZUSDT", "TRXUSDT", "VETUSDT", "CHRUSDT",
                   "ONEUSDT", "OMGUSDT", "XLMUSDT", "RUNEUSDT", "ENJUSDT",
                   "CELRUSDT", "SXPUSDT", "1INCHUSDT", "SRMUSDT", "COTIUSDT",
                   "ZECUSDT", "QTUMUSDT", "GRTUSDT", "DENTUSDT", "SFPUSDT",
                   "STORJUSDT", "IOTAUSDT", "HOTUSDT", "NEOUSDT", "COMPUSDT",
                   "LINAUSDT", "KAVAUSDT", "BATUSDT", "WAVESUSDT", "IOSTUSDT",
                   "ALPHAUSDT", "RLCUSDT", "HBARUSDT", "DODOUSDT", "XMRUSDT",
                   "SNXUSDT", "ANKRUSDT", "DASHUSDT", "REEFUSDT", "ZENUSDT",
                   "CVCUSDT", "RENUSDT", "ICXUSDT", "RSRUSDT", "SKLUSDT",
                   "NKNUSDT", "BELUSDT", "ONTUSDT", "OGNUSDT", "BLZUSDT",
                   "MTLUSDT", "CTKUSDT", "RVNUSDT", "LITUSDT", "MKRUSDT",
                   "OCEANUSDT", "ZILUSDT", "UNFIUSDT", "BANDUSDT", "YFIIUSDT",
                   "TOMOUSDT", "TRBUSDT", "KNCUSDT", "STMXUSDT", "ZRXUSDT",
                   "SCUSDT", "AKROUSDT", "XEMUSDT", "FLMUSDT", "BALUSDT",
                   "BTSUSDT", "DGBUSDT", "DEFIUSDT"]

#allowed_symbols = ['ETHUSDT']

df = big_data[(big_data.s.isin(allowed_symbols)) &
              (big_data['bnkr_%'] > 95.0) &
              (big_data['tg_%'] > 10) &
              (big_data['v'] == '5.4')
              & (big_data['long'] == True)
              & (big_data['short'] == True)
              & (big_data['ebr'] > .85)
            ]
df

,file,v,s,ebr,long,short,adg_%,bnkr_%,tg_%
26,C:\AgodaGit\passivbot\backtests/binance\ADAUSD...,5.4,ADAUSDT,0.8892,True,True,0.0697,100.0,14.49
29,C:\AgodaGit\passivbot\backtests/binance\ADAUSD...,5.4,ADAUSDT,0.8892,True,True,0.0697,100.0,14.49
62,C:\AgodaGit\passivbot\backtests/binance\ALICEU...,5.4,ALICEUSDT,0.9499,True,True,0.0752,100.0,15.95
65,C:\AgodaGit\passivbot\backtests/binance\ALICEU...,5.4,ALICEUSDT,0.9499,True,True,0.0752,100.0,15.95
149,C:\AgodaGit\passivbot\backtests/binance\CRVUSD...,5.4,CRVUSDT,0.9702,True,True,0.0517,100.0,11.02
152,C:\AgodaGit\passivbot\backtests/binance\CRVUSD...,5.4,CRVUSDT,0.9702,True,True,0.0517,100.0,11.02
164,C:\AgodaGit\passivbot\backtests/binance\DOGEUS...,5.4,DOGEUSDT,0.9827,True,True,0.0484,100.0,10.02
266,C:\AgodaGit\passivbot\backtests/binance\ICPUSD...,5.4,ICPUSDT,0.9814,True,True,0.0599,100.0,12.51
269,C:\AgodaGit\passivbot\backtests/binance\ICPUSD...,5.4,ICPUSDT,0.9814,True,True,0.0599,100.0,12.51
281,C:\AgodaGit\passivbot\backtests/binance\LINKUS...,5.4,LINKUSDT,0.9791,True,True,0.0638,100.0,13.37


In [13]:
df.drop_duplicates(subset=['s', 'tg_%', 'bnkr_%', 'ebr', 'long', 'short'], keep='first' ,inplace=True)
df

C:\Users\skhan\AppData\Local\Temp/ipykernel_53572/2147407933.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(subset=['s', 'tg_%', 'bnkr_%', 'ebr', 'long', 'short'], keep='first' ,inplace=True)


,file,v,s,ebr,long,short,adg_%,bnkr_%,tg_%
26,C:\AgodaGit\passivbot\backtests/binance\ADAUSD...,5.4,ADAUSDT,0.8892,True,True,0.0697,100.0,14.49
62,C:\AgodaGit\passivbot\backtests/binance\ALICEU...,5.4,ALICEUSDT,0.9499,True,True,0.0752,100.0,15.95
149,C:\AgodaGit\passivbot\backtests/binance\CRVUSD...,5.4,CRVUSDT,0.9702,True,True,0.0517,100.0,11.02
164,C:\AgodaGit\passivbot\backtests/binance\DOGEUS...,5.4,DOGEUSDT,0.9827,True,True,0.0484,100.0,10.02
266,C:\AgodaGit\passivbot\backtests/binance\ICPUSD...,5.4,ICPUSDT,0.9814,True,True,0.0599,100.0,12.51
281,C:\AgodaGit\passivbot\backtests/binance\LINKUS...,5.4,LINKUSDT,0.9791,True,True,0.0638,100.0,13.37
335,C:\AgodaGit\passivbot\backtests/binance\MATICU...,5.4,MATICUSDT,0.9801,True,True,0.0483,100.0,10.16
353,C:\AgodaGit\passivbot\backtests/binance\ONEUSD...,5.4,ONEUSDT,0.9808,True,True,0.0503,100.0,10.64
